# The story so far

## src/notebooks/2024-03-11

The first two notebooks showed good results, but the 3rd notebook failed to flag a culture mismatch.

## src/notebooks/2024-03-12

Tried a few iterations of the prompt, but still didn't get a culture mismatch.

## src/notebooks/2024-03-14

### self consistency

See `prediction.txt` and `prompt.txt` - this is an example of the model failing to flag a culture mismatch.

I fed these two files into Claude at https://claude.ai/chat/97542924-2249-4b78-8b3a-ce168e1b9aba
and tried my hand at "self-consistecy":
- first asking Claude if it's happy with its own prediction, which it was
- secondly, after drawing attention to the candidate's hobbies VS the company mission, Claude agreed that there was a culture mismatch

> Given this, I would likely lower the culture fit rating to something like 0.6 or 0.7 and adjust the reasoning to...

### tree of thought

https://www.promptingguide.ai/techniques/tot

See the example at the bottom of that page: "Imagine three different experts..."

I used that template to ask Claude about the value of x in the equation `2x + 5x^2 + 80 = 272`.

Claude got the wrong answer (x=6, buit discounting -6.4 as wrong).

Then I asked Claude the same question plainly:

```
What is the value of x in this equation?

2x + 5x^2 + 80 = 272
```

And Claude solved it (x=6 and x=-6.4).

Which leads me to believe that Claude already has mechanisms in place to handle self-consistency and chain/tree of thought internally.

Even if TOT worked, it would exceed the 4096 output token limit quickly.

### re-thinking the prompt

However, as I wanted to keep this experiment down to a single API call, instead of repeated "are you sure?" calls, I re-thought the prompt.

I decided to break down the scoring into "good" and "bad" parts. See below, as it makes sense.

Claude now correctly mismatches the culture fit, albeit by a low score. (I guess it's trying hard not to be biased, as per the system instruction.)

In [1]:
import os
os.getcwd()

'/home/opyate/Documents/candidate-screening-bot/app/src/notebooks/2024-03-14'

In [3]:
# help("modules")

In [4]:
from candidate_screener.main import get_match


company_name = "company1"
candidate_name = "candidate3"
prompt, response = get_match(company_name=company_name, candidate_name=candidate_name)



Company token length: 1786
Job token length: 263
Candidate token length: 1005
Prompt token length: 4014


In [5]:
print(prompt)

Please establish whether the following candidate and company are a good fit for each other, and whether the candidate is a good fit for the job.

The company's data is inside the <company></company> tag, and the job that the company is hiring for is inside the <job_description></job_description> tag.

The candidate's data is split into CV inside the <candidate_cv></candidate_cv> tag, cover letter inside the <candidate_cover_letter></candidate_cover_letter> tag, and extra text inside the <candidate_extra_text></candidate_extra_text> tag. The extra text can include any additional information about the candidate that might be relevant, and that isn't included in the CV or cover letter, like the text from the candidate's personal website's About page.

Match the candidate on each of the following attributes, with separate scores between 0 and 1 for "harmony" and "conflict": education, skills, experience, culture fit, work ethic, problem-solving abilities, leadership potential, adaptability

In [6]:
print(response[0].text)

Here is my assessment of the candidate's fit for the job and company:

<match>
education:
  harmony:
    score: 0.8
    reason: "The candidate has a strong educational background in AI and machine learning, including an AI diploma, NVidia certifications in data parallelism and transformer-based NLP, and various relevant online courses. This aligns well with the technical skills required for the role."
  conflict: 
    score: 0.1
    reason: "The job description does not specify any formal educational requirements. The candidate's education, while very relevant, may exceed what is strictly necessary for the role."
skills:
  harmony:
    score: 0.9
    reason: "The candidate has highly relevant skills in machine learning, AI, LLMs, AWS, NLP, and end-to-end software development. These directly match the key skills listed in the job description."
  conflict:
    score: 0.0
    reason: "I did not identify any skills that are misaligned with the job requirements."
experience:
  harmony:
    

# Conclusion

Finaly, culture fit gets a "conflict" score of 0.3.

Conflict scores can be weighted a little bit higher, so a "low" value like 0.3 flagged as significant.

TODO
- instruct the model to be more biased in general, and note the differences
- instruct the model to be more biased only for the soft attributes (anything other than skills, education, experience)